<a href="https://colab.research.google.com/github/Nniikkoollaass/data-science-modul-8-HW/blob/main/DS_HW_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [32]:
# для читання даних з CSV файлів
directories = {
    "walking": "data/walking",
    "running": "data/running",
    "stairs": "data/stairs",
    "idle": "data/idle"
}
data_list = []
for activity, folder_path in directories.items():
    file_names = os.listdir(folder_path)
    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        data = pd.read_csv(file_path)
        data['activity'] = activity
        data_list.append(data)
all_data = pd.concat(data_list, ignore_index=True)

In [33]:
# Функція для обчислення часових ознак
def calculate_features(data_to_analyse):
    features = {
        'mean_X': np.mean(data_to_analyse['accelerometer_X']),
        'mean_Y': np.mean(data_to_analyse['accelerometer_Y']),
        'mean_Z': np.mean(data_to_analyse['accelerometer_Z']),
        'std_X': np.std(data_to_analyse['accelerometer_X']),
        'std_Y': np.std(data_to_analyse['accelerometer_Y']),
        'std_Z': np.std(data_to_analyse['accelerometer_Z']),
        'max_X': np.max(data_to_analyse['accelerometer_X']),
        'max_Y': np.max(data_to_analyse['accelerometer_Y']),
        'max_Z': np.max(data_to_analyse['accelerometer_Z']),
        'min_X': np.min(data_to_analyse['accelerometer_X']),
        'min_Y': np.min(data_to_analyse['accelerometer_Y']),
        'min_Z': np.min(data_to_analyse['accelerometer_Z']),
        'median_X': np.median(data_to_analyse['accelerometer_X']),
        'median_Y': np.median(data_to_analyse['accelerometer_Y']),
        'median_Z': np.median(data_to_analyse['accelerometer_Z']),
    }
    return features

In [34]:
# Розрахунок ознак
feature_list = []
for activity in np.unique(all_data['activity']):
    activity_data = all_data[all_data['activity'] == activity]
    for i in range(0, len(activity_data), 200):
        current_data = activity_data[i:i+200]
        if current_data.shape[0] != 200:
            continue
        features = calculate_features(current_data)
        features['activity'] = current_data['activity'].iloc[0]
        feature_list.append(features)
# Створення нового DataFrame з розрахованими ознаками
features_df = pd.DataFrame(feature_list)
features_df.head()

# Відокремлення ознак та міток
X = features_df.drop('activity', axis=1)
y = features_df['activity']
# Розділення даних на тренувальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабування даних
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Навчання моделі SVM
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)
# Оцінка моделі SVM
svm_pred = svm_model.predict(X_test_scaled)
print("SVM Classification Report:")
print(classification_report(y_test, svm_pred))

# Навчання моделі Випадкового Лісу
rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)
# Оцінка моделі Випадкового Лісу
rf_pred = rf_model.predict(X_test_scaled)
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_pred))

SVM Classification Report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00        39
     running       1.00      1.00      1.00        38
      stairs       1.00      1.00      1.00         3
     walking       1.00      1.00      1.00        28

    accuracy                           1.00       108
   macro avg       1.00      1.00      1.00       108
weighted avg       1.00      1.00      1.00       108

Random Forest Classification Report:
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00        39
     running       1.00      1.00      1.00        38
      stairs       1.00      1.00      1.00         3
     walking       1.00      1.00      1.00        28

    accuracy                           1.00       108
   macro avg       1.00      1.00      1.00       108
weighted avg       1.00      1.00      1.00       108

